In [1]:
#library(readr)
library(ggplot2)
library(tableone)
library(epiDisplay)
print(getwd())

Warning message:
"package 'ggplot2' was built under R version 3.2.5"Warning message:
"package 'tableone' was built under R version 3.2.5"Warning message:
"package 'epiDisplay' was built under R version 3.2.5"Loading required package: foreign
Loading required package: survival
Loading required package: MASS
Loading required package: nnet

Attaching package: 'epiDisplay'

The following object is masked from 'package:ggplot2':

    alpha



[1] "C:/Users/Lyx/Documents/Jupyter Notebooks"


In [2]:
#read data from csv
df <- read.csv('D:/p/acr/acr.csv',fileEncoding = "UTF-8-BOM")

In [3]:
#filter, exclude those without y4_p and make sure all the acr is <30
df <- df[which(!is.na(df$y4_p) & df$MA_UCREA<30),]

In [4]:
#converting categorical variables to factors
cols <- c("SEX", "smoke", "drink", "hyp", "chd", "hlip", "dm", "stroke", "y4_p", "hypdrug", "dmdrug", "hlipdrug", "CKD_grade")
df[cols] <- lapply(df[cols], factor)
#df$SEX <- factor(df$SEX)

In [5]:
#insert a quantile variable for ACR
quantile(df$MA_UCREA,na.rm = TRUE)

0%    25%    50%    75%   100% 
 0.000  4.020  5.750  9.185 29.860

In [6]:
attach(df)
df$acrquan[MA_UCREA >= 0 &  MA_UCREA < 4.02] <- 0
df$acrquan[MA_UCREA >= 4.02 &  MA_UCREA < 5.75] <- 1
df$acrquan[MA_UCREA >= 5.75 &  MA_UCREA < 9.185] <- 2
df$acrquan[MA_UCREA >= 9.185 &  MA_UCREA < 30] <- 3

## Definitions for creating new variables based on the exist ones

### 1. Current smoking (c_smoke)
> Current smoking was defined as smoking one cigarette or more per day for at least half a year.

if (smoke == 1 | smoke ==2) then c_smoke = 0

else if (smoke ==3) then c_smoke = 1
### 2. Current drinking (c_drink)
> Current drinking was defined as drinking once per week for at least half a year.

if (drink == 1 | drink ==2) then c_drink = 0

else if (drink ==3) then c_drink = 1
### 3. Hypertension (d_hyp)
> Hypertension was defined as any self-reported history of hypertension, a systolic blood pressure (SBP) ≥140 mmHg, a diastolic blood pressure (DBP) ≥90 mmHg, or usage of any antihypertensive drugs.

if (hyp==1 | SBP>=140 | DBP >=90 | hypdrug==1) then d_hyp = 1

else d_hyp = 0
### 4. Diabetes (d_dm)
>Diabetes mellitus was defined as any self-reported history of diabetes, fasting blood glucose (FBG) ≥7.0 mmol/L and a 2-hour plasma glucose value in oral glucose tolerance test (OGTT) ≥11.1 mmol/L, or usage of any hypoglycemic drugs.

if (dm==1 | GLU >=7 | dmdrug==1) then d_dm = 1

else d_dm = 0
### 5. Dyslipidemia (d_hlip)
>Dyslipidemia was defined as any self-reported history of dyslipidemia, triglyceride (TG) ≥1.7 mmol/L (150 mg/dL), total cholesterol (TC) ≥5.18 mmol/L (200 mg/dL), low-density lipoprotein (LDL-C) ≥3.37 mmol/L (130 mg/dL), high-density lipoprotein (HDL-C) <1.04 mmol/L (40 mg/dL), or usage of any lipid-lowering drugs.

if (hlip==1 | TG >= 1.7 | TCHO >= 5.18 | LDL_C >=3.37 | HDL_C < 1.04 | hlipdrug==1) then d_hlip =1

else d_hlip =0
### 6. CVD (d_cvd)
>Cardiovascular disease (CVD) was defined as any self-reported history of coronary heart disease or stroke.

if (chd==1 | stroke==1) then d_cvd = 1

else d_cvd = 0

In [7]:
#inserting c_smoke, c_drink, d_hyp, d_dm, d_hlip and d_cvd
df$c_smoke <- ifelse((smoke==1 | smoke ==2), 0, 1)
df$c_drink <- ifelse((drink==1 | drink ==2), 0, 1)
df$d_hyp <- ifelse((hyp==1 | SBP>=140 | DBP >=90 | hypdrug==1), 1, 0)
df$d_dm <- ifelse((dm==1 | GLU >=7 | dmdrug==1), 1, 0)
df$d_hlip <- ifelse((hlip==1 | TG >= 1.7 | TCHO >= 5.18 | LDL_C >=3.37 | HDL_C < 1.04 | hlipdrug==1), 1, 0)
df$d_cvd <- ifelse((chd==1 | stroke==1), 1, 0)
df$lnacr <- ifelse((MA_UCREA==0), 0, log(MA_UCREA))
detach(df)
#converting into factors
#converting categorical variables to factors
cols <- c("c_smoke", "c_drink", "d_hyp", "d_dm", "d_hlip", "d_cvd", "acrquan")
df[cols] <- lapply(df[cols], factor)

In [8]:
summary(df)

     NO2014         SEX           age            BMI            SBP01      
 Min.   :20140001   1:1245   Min.   :41.0   Min.   :16.03   Min.   : 77.0  
 1st Qu.:20141012   2:2262   1st Qu.:53.0   1st Qu.:23.61   1st Qu.:116.0  
 Median :20141969            Median :58.0   Median :25.61   Median :127.0  
 Mean   :20141969            Mean   :58.8   Mean   :25.91   Mean   :128.4  
 3rd Qu.:20142940            3rd Qu.:63.0   3rd Qu.:28.03   3rd Qu.:140.0  
 Max.   :20143907            Max.   :90.0   Max.   :53.63   Max.   :194.0  
                                            NA's   :2       NA's   :5      
     DBP01            SBP02           DBP02            SBP03      
 Min.   : 36.00   Min.   : 79.0   Min.   : 40.00   Min.   : 83.0  
 1st Qu.: 68.00   1st Qu.:114.0   1st Qu.: 68.00   1st Qu.:113.0  
 Median : 74.00   Median :124.0   Median : 74.00   Median :123.0  
 Mean   : 74.91   Mean   :125.7   Mean   : 74.33   Mean   :124.3  
 3rd Qu.: 81.00   3rd Qu.:136.0   3rd Qu.: 80.00   3rd Qu

## Table 1

Using tableone package, create table 1 stratified by acrquan of ACR.

In [9]:
#print table1
#listVars <- c("age", "SEX", "BMI", "SBP", "DBP", "CREA", "GLU", "MA_UCREA", "hyp")
listVars <- c("age", "BMI", "CREA", "MA_UCREA", "SEX", "c_smoke", "c_drink", "d_hyp", "d_dm", "d_hlip", "d_cvd", "CKD_grade", "hypdrug", "dmdrug", "hlipdrug", "y4_p")
#Define categorical variables
catVars <- c("SEX","c_smoke", "c_drink", "d_hyp", "d_dm", "d_hlip", "d_cvd", "CKD_grade", "hypdrug", "dmdrug", "hlipdrug")
table1 <- CreateTableOne(listVars, df, catVars, strata = c("acrquan"))
print(table1)

                      Stratified by acrquan
                       0             1             2             3            
  n                      871           881           878           877        
  age (mean (sd))      58.42 (7.86)  58.04 (8.23)  58.92 (8.11)  59.83 (9.06) 
  BMI (mean (sd))      25.56 (3.31)  25.69 (3.43)  25.96 (3.54)  26.44 (3.72) 
  CREA (mean (sd))     91.81 (12.91) 88.35 (13.10) 86.76 (12.52) 87.27 (14.47)
  MA_UCREA (mean (sd))  2.75 (1.08)   4.85 (0.51)   7.20 (1.01)  14.93 (5.13) 
  SEX = 2 (%)            486 (55.8)    539 (61.2)    610 (69.5)    627 (71.5) 
  c_smoke = 1 (%)        146 (16.8)    153 (17.4)    122 (13.9)    135 (15.5) 
  c_drink = 1 (%)        144 (16.6)    124 (14.1)    108 (12.3)     97 (11.1) 
  d_hyp = 1 (%)          320 (36.8)    336 (38.2)    391 (44.6)    510 (58.4) 
  d_dm = 1 (%)           140 (16.1)    128 (14.6)    158 (18.1)    230 (26.4) 
  d_hlip = 1 (%)         654 (75.2)    641 (73.1)    664 (75.7)    696 (79.5) 
  d_cvd 

In [10]:
#print a overall table1 
table1 <- CreateTableOne(vars=listVars, data=df, factorVars=catVars)
print(table1)

                      
                       Overall      
  n                     3507        
  age (mean (sd))      58.80 (8.35) 
  BMI (mean (sd))      25.91 (3.52) 
  CREA (mean (sd))     88.54 (13.41)
  MA_UCREA (mean (sd))  7.44 (5.33) 
  SEX = 2 (%)           2262 (64.5) 
  c_smoke = 1 (%)        556 (15.9) 
  c_drink = 1 (%)        473 (13.5) 
  d_hyp = 1 (%)         1557 (44.5) 
  d_dm = 1 (%)           656 (18.8) 
  d_hlip = 1 (%)        2655 (75.9) 
  d_cvd = 1 (%)          463 (13.2) 
  CKD_grade (%)                     
     1                   289 ( 8.3) 
     2                  2917 (83.5) 
     3                   288 ( 8.2) 
  hypdrug = 1 (%)        872 (24.9) 
  dmdrug = 1 (%)         308 ( 8.8) 
  hlipdrug = 1 (%)       337 ( 9.6) 
  y4_p = 1 (%)          1959 (55.9) 


In [11]:
#crude: Non-adjusted
#Model 2: adjust for: SEX; age
model2_ACR = glm(formula= y4_p ~ MA_UCREA + SEX + age, data=df, family=binomial("logit"),na.action = na.exclude)
logistic.display(model2_ACR, decimal = 3)$table

,crude OR(95%CI),adj. OR(95%CI),P(Wald's test),P(LR-test)
MA_UCREA (cont. var.),"1.051 (1.036,1.065)","1.055 (1.04,1.07)",< 0.001,< 0.001
,,,,
SEX: 2 vs 1,"0.48 (0.415,0.554)","0.497 (0.426,0.58)",< 0.001,< 0.001
,,,,
age (cont. var.),"1.1 (1.089,1.111)","1.097 (1.085,1.108)",< 0.001,< 0.001
,,,,


In [12]:
model2_lnACR = glm(formula= y4_p ~ lnacr + SEX + age, data=df, family=binomial("logit"),na.action = na.exclude)
logistic.display(model2_lnACR, decimal = 3)$table

,crude OR(95%CI),adj. OR(95%CI),P(Wald's test),P(LR-test)
lnacr (cont. var.),"1.415 (1.292,1.551)","1.499 (1.357,1.656)",< 0.001,< 0.001
,,,,
SEX: 2 vs 1,"0.48 (0.415,0.554)","0.494 (0.423,0.577)",< 0.001,< 0.001
,,,,
age (cont. var.),"1.1 (1.089,1.111)","1.098 (1.086,1.109)",< 0.001,< 0.001
,,,,


In [13]:
model2 = glm(formula= y4_p ~ acrquan + SEX + age, data=df, family=binomial("logit"),na.action = na.exclude)
logistic.display(model2, decimal = 3)$table

,crude OR(95%CI),adj. OR(95%CI),P(Wald's test),P(LR-test)
acrquan: ref.=0,,,,< 0.001
1,"1.228 (1.018,1.481)","1.378 (1.125,1.688)",0.002,
2,"1.461 (1.209,1.764)","1.641 (1.338,2.013)",< 0.001,
3,"1.963 (1.621,2.378)","2.198 (1.783,2.709)",< 0.001,
,,,,
SEX: 2 vs 1,"0.48 (0.415,0.554)","0.486 (0.416,0.569)",< 0.001,< 0.001
,,,,
age (cont. var.),"1.1 (1.089,1.111)","1.097 (1.086,1.108)",< 0.001,< 0.001
,,,,


In [14]:
#Model 3: adjust for: SEX; age; BMI; c_smoke; c_drink; CREA; d_hyp; d_dm; d_hlip; d_cvd; CKD_grade; hypdrug; dmdrug; hlipdrug; 
model3_ACR = glm(formula= y4_p ~ MA_UCREA + SEX + age + BMI + c_smoke + c_drink + CREA + d_hyp + d_dm + d_hlip + d_cvd + CKD_grade + hypdrug + dmdrug + hlipdrug, data=df, family=binomial("logit"),na.action = na.exclude)
logistic.display(model3_ACR, decimal = 3)$table

,crude OR(95%CI),adj. OR(95%CI),P(Wald's test),P(LR-test)
MA_UCREA (cont. var.),"1.05 (1.036,1.065)","1.047 (1.031,1.063)",< 0.001,< 0.001
,,,,
SEX: 2 vs 1,"0.473 (0.409,0.547)","0.629 (0.482,0.821)",< 0.001,< 0.001
,,,,
age (cont. var.),"1.101 (1.09,1.112)","1.086 (1.074,1.098)",< 0.001,< 0.001
,,,,
BMI (cont. var.),"1.018 (0.998,1.037)","0.986 (0.965,1.008)",0.2068,0.2069
,,,,
c_smoke: 1 vs 0,"1.499 (1.242,1.81)","1.201 (0.933,1.545)",0.1546,0.154
,,,,


In [15]:
model3_lnACR = glm(formula= y4_p ~ lnacr + SEX + age + BMI + c_smoke + c_drink + CREA + d_hyp + d_dm + d_hlip + d_cvd + CKD_grade + hypdrug + dmdrug + hlipdrug, data=df, family=binomial("logit"),na.action = na.exclude)
logistic.display(model3_lnACR, decimal = 3)$table

,crude OR(95%CI),adj. OR(95%CI),P(Wald's test),P(LR-test)
lnacr (cont. var.),"1.41 (1.287,1.546)","1.432 (1.291,1.587)",< 0.001,< 0.001
,,,,
SEX: 2 vs 1,"0.473 (0.409,0.547)","0.636 (0.487,0.83)",< 0.001,< 0.001
,,,,
age (cont. var.),"1.101 (1.09,1.112)","1.086 (1.074,1.098)",< 0.001,< 0.001
,,,,
BMI (cont. var.),"1.018 (0.998,1.037)","0.985 (0.964,1.007)",0.188,0.1881
,,,,
c_smoke: 1 vs 0,"1.499 (1.242,1.81)","1.201 (0.933,1.547)",0.1544,0.1539
,,,,


In [16]:
model3 = glm(formula= y4_p ~ acrquan + SEX + age + BMI + c_smoke + c_drink + CREA + d_hyp + d_dm + d_hlip + d_cvd + CKD_grade + hypdrug + dmdrug + hlipdrug, data=df, family=binomial("logit"),na.action = na.exclude)
logistic.display(model3, decimal = 3)$table

,crude OR(95%CI),adj. OR(95%CI),P(Wald's test),P(LR-test)
acrquan: ref.=0,,,,< 0.001
1,"1.229 (1.019,1.484)","1.42 (1.153,1.748)",< 0.001,
2,"1.472 (1.218,1.778)","1.621 (1.313,2)",< 0.001,
3,"1.957 (1.614,2.372)","2.036 (1.636,2.534)",< 0.001,
,,,,
SEX: 2 vs 1,"0.473 (0.409,0.547)","0.625 (0.478,0.816)",< 0.001,< 0.001
,,,,
age (cont. var.),"1.101 (1.09,1.112)","1.085 (1.074,1.098)",< 0.001,< 0.001
,,,,
BMI (cont. var.),"1.018 (0.998,1.037)","0.985 (0.963,1.007)",0.168,0.168


In [17]:
#interaction with smoke
a<-glm(formula= y4_p ~ lnacr *c_smoke + age + BMI + SEX + c_drink + CREA + d_hyp + d_dm + d_hlip + d_cvd + CKD_grade + hypdrug + dmdrug + hlipdrug, data=df, family=binomial)
logistic.display(a)$table

,crude OR(95%CI),adj. OR(95%CI),P(Wald's test),P(LR-test)
lnacr (cont. var.),"1.41 (1.29,1.55)","1.43 (1.28,1.6)",< 0.001,1
,,,,
c_smoke: 1 vs 0,"1.5 (1.24,1.81)","1.2 (0.71,2.03)",0.505,0.506
,,,,
age (cont. var.),"1.1 (1.09,1.11)","1.09 (1.07,1.1)",< 0.001,< 0.001
,,,,
BMI (cont. var.),"1.02 (1,1.04)","0.99 (0.96,1.01)",0.188,0.188
,,,,
SEX: 2 vs 1,"0.47 (0.41,0.55)","0.64 (0.49,0.83)",< 0.001,< 0.001
,,,,


In [18]:
#ggplot(df, aes(x=MA_UCREA)) + 
#    geom_histogram()

In [19]:
#ggplot(data = df)+ 
#  geom_point(mapping = aes(x = MA_UCREA, y = SBP))+ 
#  facet_grid(y4_p~ CKD_grade)

In [20]:
#ggplot(data = df)+ 
#  geom_smooth(mapping = aes(x = MA_UCREA, y = SBP))

In [21]:
names(df)

[1] "NO2014"    "SEX"       "age"       "BMI"       "SBP01"     "DBP01"    
 [7] "SBP02"     "DBP02"     "SBP03"     "DBP03"     "SBP"       "DBP"      
[13] "CREA"      "GLU"       "HDL_C"     "LDL_C"     "TCHO"      "TG"       
[19] "MA"        "MA_UCREA"  "UCREA"     "smoke"     "drink"     "hyp"      
[25] "chd"       "hlip"      "dm"        "stroke"    "y4_p"      "hypdrug"  
[31] "dmdrug"    "hlipdrug"  "eGFR"      "CKD_grade" "hyp_new"   "acrquan"  
[37] "c_smoke"   "c_drink"   "d_hyp"     "d_dm"      "d_hlip"    "d_cvd"    
[43] "lnacr"

In [22]:
#ggplot(data = df) + 
#  geom_bar(mapping = aes(x = CKD_grade, fill = smoke),position="dodge")

In [23]:
#ggplot(data = df, mapping = aes(x = age, y = SBP, group=CKD_grade)) + 
#  geom_boxplot() +
#  coord_flip()